**Please only run the following cell if you're running this notebook on Google colab or another cloud hosted notebook server.**

In [0]:
!pip install qiskit

# B92 Quantum Key Distribution (QKD) Protocol

This notebook is a _demonstration_ of the B92 Protocol for QKD using Qiskit. 
B92is a quantum key distribution scheme developed by Charles Bennett in 1992 as an improvement over BB84 ([paper]).
The first two pages of the paper are readable and should give you all the necessary information required. 

[paper]: https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.68.3121

$$ \newcommand{\ket}[1]{\left|{#1}\right\rangle}
\newcommand{\bra}[1]{\left\langle{#1}\right|}$$

In [0]:
from qiskit import BasicAer
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import execute
from qiskit import IBMQ


from qiskit.tools.monitor import job_monitor
%pylab inline

First we will simulate a noiseless quantum channel free from an eavesdropper. The qubit capabilites of the available backends are listed below:


| Backend | Number of qubits |
|----------|------------------|
|qasm_simulator|24|
|ibmq_qasm_simulator|32|
|ibmqx4|5|
|ibmqx2|5|
|ibmq_16_melbourne|14|

## Choosing bases and encoding states

Alice generates one binary string and encodes her qubits using the following scheme:

$0  \rightarrow \ket{ 0 }$

$1  \rightarrow \ket{ + }$ 

Bob also generates a binary string and uses the following convention to choose a basis for measurement

$0 \rightarrow$ Hadamard basis

$1 \rightarrow$ Computational basis



In [0]:
num_qubits = 32

alice_state = np.random.randint(2, size=num_qubits)
bob_basis = np.random.randint(2, size=num_qubits)


print(alice_state)
print(bob_basis)

## Creating the circuit

Based on the following result:

$H\ket{0} = \ket{+}$

Our algorithm to construct the circuit is as follows:

1. Whenever Alice wants to encode 1 in a qubit, she applies an $H$ gate to the qubit. To encode 0, no action is needed.

2. She then _sends_ the qubits to Bob (symbolically represented in this circuit using wires)

3. Bob measures the qubits according to his binary string. To measure a qubit in the Hadamard basis, he applies an $H$ gate to the corresponding qubit and then performs a mesurement on the computational basis. 



In [0]:
q = QuantumRegister(num_qubits,'q')
c = ClassicalRegister(num_qubits,'c')

b92 = QuantumCircuit(q,c,name='b92')

# Alice prepares her qubits
for index, _ in enumerate(alice_state):
    if alice_state[index] == 1:
        b92.h(q[index])
b92.barrier()  

# Bob measures the received qubits
for index, _ in enumerate(bob_basis):
    if bob_basis[index] == 0:
        b92.h(q[index])
        
b92.barrier()        
b92.measure(q,c)
    
            
# b92.draw()


### Running it on the IBMQ machines

The following cell is only needed if you want to use the IBM Q Experience devices. Please enter your API token between the quotes. Instructions are given [here](https://github.com/Qiskit/qiskit/blob/master/docs/install.rst#access-ibm-q-devices)
Uncomment the code in the cell to use it. 

In [0]:
# token = ''
# IBMQ.enable_account(token)
# IBMQ.backends()

## Performing the actual experiment

Run the code on the desired backend. Use the cell above to authenticate yourself to the IBMQ network.

In [0]:
# backend = BasicAer.get_backend('qasm_simulator')
# backend = IBMQ.get_backend('ibmq_16_melbourne')
backend = IBMQ.get_backend('ibmq_qasm_simulator')

job = execute(b92,backend,shots=8192)

job_monitor(job)

result = job.result()
counts = result.get_counts()

# Prints out all the measured binary strings by Bob (reversed)
for count in [*counts]:
    print(count) 

## Creating the key

Alice and bob keep only those bits for which bob's measurement outcome was 1.

In [0]:
alice_keys = []
bob_keys = []

# print("Bob's key\t\tAlice's key")
for count in [*counts]:
    bob_measurement = count[::-1] # qubit orders are reversed in qiskit
    alice_key = ''
    bob_key = ''
    for i in range(num_qubits):
        if bob_measurement[i] == '1': # Only choose bits where Bob measured a 1
            alice_key += str(alice_state[i])
            bob_key += str(bob_basis[i])
    bob_keys.append(bob_key)
    alice_keys.append(alice_key)
    print(f'{bob_key}\t\t{alice_key}')


In [0]:
# Average length of a key

(np.mean([len(key) for key in bob_keys]))



In [0]:
# Average fraction of 0s and 1s in a key
np.mean([ [key.count('0')/len(key), key.count('1')/len(key)] for key in bob_keys],axis=0)